In [12]:
import pandas as pd
import os
# auxiliary funtions
from utils import syncing
from utils.rotation_camera_evaluation import mapping_df

### LOAD DATA

In [8]:
DATA_DIR = 'data/nfl-health-and-safety-helmet-assignment/'

In [9]:
n_test_videos = len(os.listdir(DATA_DIR+'test/'))
# Run in debug mode unless during submission
if n_test_videos == 6:
    debug = True
else:
    debug = False
print(f"debug = {debug}")

debug = True


In [10]:
# Configurables
n_debug_samples = 1
random_state = 2020

In [5]:
# Load data
labels = pd.read_csv(f'{DATA_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{DATA_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{DATA_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{DATA_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{DATA_DIR}/test_baseline_helmets.csv')
    
tracking = syncing.add_track_features(tracking)

In [6]:
if debug:
    helmets = syncing.add_cols(helmets)
    labels = syncing.add_cols(labels)
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates() \
        .sample(n_debug_samples, random_state=random_state).tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]

print(f'tracking.shape = {tracking.shape}')
print(f'helmets.shape = {helmets.shape}')
print(f'labels.shape = {labels.shape}')

tracking.shape = (7568, 18)
helmets.shape = (8879, 8)
labels.shape = (6254, 15)


In [14]:
submission_df = multiple_mapping(tracking, helmets)
submission_df.to_csv('submission-baseline.csv', index=False)

  0%|          | 0/354 [00:00<?, ?it/s]


TypeError: mapping_df() missing 1 required positional argument: 'tracking'